In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Datasets/Input/AQI.csv')
data.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [8]:
# splitting the data
x = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [9]:
#Independent data
x.head()

,T,TM,Tm,SLP,H,VV,V,VM
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2


In [10]:
# target
y.head()

0    219.720833
1    182.187500
2    154.037500
3    223.208333
4    200.645833
Name: PM 2.5, dtype: float64

In [ ]:
# keras-tuner docs
    # Examples
        # https://keras.io/keras_tuner/
        # https://keras.io/guides/keras_tuner/getting_started/
        # https://github.com/keras-team/keras-io/blob/master/guides/keras_tuner/getting_started.py
    # Documentations
        # https://keras.io/api/keras_tuner/

In [5]:
# Installing keras-tuner
! pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.6 MB/s eta 0:00:00


In [6]:
# importing keras modules
import keras
from keras.models import Sequential
from keras.layers import Dense
import keras_tuner

In [18]:
def build_model(hp):
    # model
    model = Sequential()

    # Input layer
    model.add(Dense(units=8, kernel_initializer='he_uniform', input_dim=8))

    # hidden layer hp
    for i in range(hp.Int('num_layer', 2, 20)):
        model.add(Dense(units=hp.Int(f'Units_{i}',
                                     min_value=32,
                                     max_value=512,
                                     step=32),
                        activation=hp.Choice("activation", ["relu", "tanh", "sigmoid"])))

    # output layer
    model.add(Dense(1, activation='relu'))

    # compilation
    model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss = 'mean_squared_error', metrics=['accuracy'])

    return model

In [19]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="AQI",
)

In [20]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
num_layer (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
Units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
Units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [21]:
# data splitting
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state = 42)

In [22]:
tuner.search(X_train, y_train, epochs=5, validation_data = (X_test, y_test))

Trial 3 Complete [00h 00m 15s]
val_accuracy: 0.007621951401233673

Best val_accuracy So Far: 0.015243902802467346
Total elapsed time: 00h 00m 35s


In [26]:
# Model summary
tuner.results_summary()

Results summary
Results in my_dir/AQI
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
num_layer: 11
Units_0: 288
activation: relu
Units_1: 64
learning_rate: 0.001
Units_2: 32
Units_3: 32
Units_4: 32
Units_5: 32
Units_6: 32
Units_7: 32
Units_8: 32
Units_9: 32
Units_10: 32
Score: 0.015243902802467346

Trial 1 summary
Hyperparameters:
num_layer: 4
Units_0: 96
activation: sigmoid
Units_1: 32
learning_rate: 0.001
Units_2: 256
Units_3: 448
Units_4: 256
Units_5: 480
Units_6: 160
Units_7: 448
Units_8: 224
Units_9: 384
Units_10: 160
Score: 0.007621951401233673

Trial 2 summary
Hyperparameters:
num_layer: 14
Units_0: 192
activation: relu
Units_1: 288
learning_rate: 0.0001
Units_2: 448
Units_3: 288
Units_4: 32
Units_5: 320
Units_6: 288
Units_7: 512
Units_8: 96
Units_9: 224
Units_10: 288
Units_11: 32
Units_12: 32
Units_13: 32
Score: 0.007621951401233673


In [25]:
# Best model out of top two
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.build(input_shape=(None, 28, 28))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 288)               2592      
                                                                 
 dense_2 (Dense)             (None, 64)                18496     
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 32)                1056      
                                                                 
 dense_5 (Dense)             (None, 32)                1056      
                                                                 
 dense_6 (Dense)             (None, 32)                1